# Declaration of Libraries

In [53]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.metrics import multilabel_confusion_matrix
from matplotlib import pyplot as plt
from sklearn import datasets
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn import tree

In [26]:
spotify_data = pd.read_csv("/content/spotify_data.csv")
spotify_data = spotify_data[spotify_data.columns[1:]]

In [ ]:
spotify_data.tail()

,artist_name,track_name,track_id,popularity,year,genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
1159759,Nicola Conte,Black Spirits,0m27F0IGHLGAWhqd6ccYst,4,2011,trip-hop,0.373,0.742,10,-6.453,0,0.0736,0.325,0.000141,0.1590,0.5220,107.951,344013,3
1159760,Nicola Conte,Quiet Dawn,6er9p611eHEcUCU50j7D57,3,2011,trip-hop,0.516,0.675,7,-7.588,0,0.0326,0.788,0.000129,0.1300,0.2640,119.897,285067,4
1159761,Amon Tobin,Morning Ms Candis,7jsMMqxy1tt0rH5FzYcZTQ,2,2011,trip-hop,0.491,0.440,5,-8.512,1,0.0274,0.477,0.003130,0.0936,0.0351,100.076,214253,4
1159762,Peace Orchestra,Happy Christmas (War Is Over),77lA1InUaXztuRk2vOzD1S,0,2011,trip-hop,0.480,0.405,0,-13.343,1,0.0276,0.431,0.000063,0.1250,0.2020,133.885,239133,3
1159763,Mo' Horizons,Hit the Road Jack (Pé Na Éstrada),4oMiOwhDZEdBuzAfhzRHbi,3,2011,trip-hop,0.782,0.861,1,-7.292,0,0.1250,0.220,0.000008,0.0581,0.8570,89.987,212227,4


## Data Preprocessing

In [27]:
spotify_data.isna().any()
#spotify_data.loc[spotify_data.duration_ms.isna(), :]

artist_name         False
track_name          False
track_id            False
popularity           True
year                 True
genre                True
danceability         True
energy               True
key                  True
loudness             True
mode                 True
speechiness          True
acousticness         True
instrumentalness     True
liveness             True
valence              True
tempo                True
duration_ms          True
time_signature       True
dtype: bool

In [28]:
spotify_data.loc[spotify_data.duration_ms.isna(), :]

,artist_name,track_name,track_id,popularity,year,genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
153033,Grupo ExQuadrilha,Terminais da Despedida,2EFTmVXGF8z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
#spotify_data = spotify_data.iloc[spotify_data.index != 1011592].reset_index(drop=True)
#spotify_data = spotify_data.iloc[spotify_data.index != 1018401].reset_index(drop=True)
spotify_data = spotify_data.loc[:, spotify_data.columns != "track_id"]

#Changed Instrumentalness so that it is not the proportion of of the track that have vocals but rather seperating music into vocals and no vocals
spotify_data["instrumentalness"] = spotify_data["instrumentalness"].apply(lambda x: False if x == 0 else True)

<ipython-input-29-206af01a9a59>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spotify_data["instrumentalness"] = spotify_data["instrumentalness"].apply(lambda x: False if x == 0 else True)


In [30]:
spotify_data.isna().any()

artist_name         False
track_name          False
popularity           True
year                 True
genre                True
danceability         True
energy               True
key                  True
loudness             True
mode                 True
speechiness          True
acousticness         True
instrumentalness    False
liveness             True
valence              True
tempo                True
duration_ms          True
time_signature       True
dtype: bool

In [31]:
spotify_data.dropna(inplace = True)

In [32]:
spotify_data.isna().any()

artist_name         False
track_name          False
popularity          False
year                False
genre               False
danceability        False
energy              False
key                 False
loudness            False
mode                False
speechiness         False
acousticness        False
instrumentalness    False
liveness            False
valence             False
tempo               False
duration_ms         False
time_signature      False
dtype: bool

In [33]:
spotify_data.describe()

,popularity,year,danceability,energy,key,loudness,mode,speechiness,acousticness,liveness,valence,tempo,duration_ms,time_signature
count,153033.000000,153033.000000,153033.000000,153033.000000,153033.000000,153033.000000,153033.000000,153033.000000,153033.000000,153033.000000,153033.000000,153033.000000,1.530330e+05,153033.000000
mean,17.625048,2012.936112,0.532671,0.657644,5.274542,-8.523489,0.629498,0.093016,0.305273,0.226409,0.448215,122.046859,2.565516e+05,3.890880
std,14.328150,0.805097,0.181771,0.265450,3.557943,5.357510,0.482941,0.130906,0.350476,0.203650,0.268216,29.383314,1.579455e+05,0.454852
min,0.000000,2012.000000,0.000000,0.000000,0.000000,-56.098000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.027000e+03,0.000000
25%,6.000000,2012.000000,0.411000,0.479000,2.000000,-10.329000,0.000000,0.037000,0.004590,0.097700,0.219000,99.989000,1.891070e+05,4.000000
50%,15.000000,2013.000000,0.543000,0.718000,5.000000,-7.064000,1.000000,0.050400,0.118000,0.136000,0.425000,122.428000,2.334800e+05,4.000000
75%,27.000000,2014.000000,0.669000,0.883000,8.000000,-4.991000,1.000000,0.088800,0.610000,0.299000,0.664000,139.977000,2.945870e+05,4.000000
max,91.000000,2014.000000,0.991000,1.000000,11.000000,6.172000,1.000000,0.969000,0.996000,1.000000,1.000000,248.221000,6.000461e+06,5.000000


In [15]:
spotify_data[spotify_data.duplicated()]

,artist_name,track_name,popularity,year,genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature


# Classification Method


## Correlation of each attribute

## Training the Model

# KNN


In [ ]:
# Split the dataset into features (X) and target variable (y)
#X = spotify_data[['popularity', 'danceability', 'energy', 'key']].reset_index(drop=True)
X = spotify_data[['popularity', 'danceability', 'energy', 'key']].reset_index(drop=True)
y = spotify_data['genre'].reset_index(drop=True)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Feature Scaling: Making sure the data is not skewed by large numbers
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.fit_transform(X_test)

In [ ]:
#Find out how many neighbours is needed
import math
sqrt_of_y = round(math.sqrt(len(y_test)))
n_number = sqrt_of_y - 1 if sqrt_of_y % 2 == 0 else sqrt_of_y
print( n_number )

481


In [ ]:
# Create and train the KNN classifier
knn = KNeighborsClassifier(n_neighbors = n_number, p=2, metric='euclidean')  # Set the number of neighbors (k) as desired
knn.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = knn.predict(X_test)

# Evaluating KNN : K-Fold Cross Validation



In [ ]:
# Evaluate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Predict the genre of new tracks
new_track = [[80, 0.7, 0.8, 5]]  # Provide the attributes (popularity, danceability, energy, key) of the new track
predicted_genre = knn.predict(new_track)
print(f"Predicted Genre: {predicted_genre}")

Accuracy: 0.13363483119425057
Predicted Genre: ['pop']


In [ ]:
#Feature Scaling
X = sc_X.transform(X)

In [ ]:
#Cross Validation
cross_val_score(knn, X, y)

array([0.13026777, 0.13098774, 0.12334826, 0.10958686, 0.12287887])

# **DECISION TREE**

In [39]:
#number of unique labels
len(spotify_data['genre'].drop_duplicates())

82

In [40]:
spotify_data


,artist_name,track_name,popularity,year,genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Jason Mraz,I Won't Give Up,68.0,2012.0,acoustic,0.483,0.303,4.0,-10.058,1.0,0.0429,0.6940,False,0.1150,0.139,133.406,240166.0,3.0
1,Jason Mraz,93 Million Miles,50.0,2012.0,acoustic,0.572,0.454,3.0,-10.286,1.0,0.0258,0.4770,True,0.0974,0.515,140.182,216387.0,4.0
2,Joshua Hyslop,Do Not Let Me Go,57.0,2012.0,acoustic,0.409,0.234,3.0,-13.711,1.0,0.0323,0.3380,True,0.0895,0.145,139.832,158960.0,4.0
3,Boyce Avenue,Fast Car,58.0,2012.0,acoustic,0.392,0.251,10.0,-9.845,1.0,0.0363,0.8070,False,0.0797,0.508,204.961,304293.0,4.0
4,Andrew Belle,Sky's Still Blue,54.0,2012.0,acoustic,0.430,0.791,6.0,-5.419,0.0,0.0302,0.0726,True,0.1100,0.217,171.864,244320.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153028,Arlindo Cruz,Isso é Felicidade,23.0,2014.0,samba,0.655,0.687,2.0,-8.214,1.0,0.1150,0.7430,True,0.2290,0.793,86.173,176507.0,4.0
153029,Arlindo Cruz,Ilicitação,11.0,2014.0,samba,0.704,0.816,2.0,-7.164,1.0,0.1010,0.6430,False,0.1680,0.898,91.976,214733.0,4.0
153030,Arlindo Cruz,O Surfista e o Sambista (feat. Pedro Scooby),17.0,2014.0,samba,0.653,0.756,2.0,-6.217,1.0,0.2500,0.5960,False,0.2800,0.884,99.830,203467.0,4.0
153031,Zé Paulo Sierra,"Nas Veias Do Brasil, É A Viradouro Em Um Dia D...",16.0,2014.0,samba,0.585,0.868,7.0,-6.187,0.0,0.1170,0.5290,True,0.6890,0.874,139.209,265280.0,4.0


In [34]:
# split dataset in features and target variables
feature_cols = ['popularity', 'danceability', 'energy', 'key']
X = spotify_data[feature_cols]
y = spotify_data.genre

In [35]:
#splitting dataset into training and test set
X_train, X_test, y_train, y_test  = train_test_split(X, y, test_size = 0.3, random_state = 1) # 70/30

In [36]:
spotify_data.year.isna().sum()


0

In [41]:
#Decision tree classifier object
clf = DecisionTreeClassifier()

# Train Decision tree Classifier
clf = clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)


In [42]:
accuracy_score(y_test, y_pred, normalize = True)

0.08540622957961229

**EVALUATION**

In [43]:
cm = multilabel_confusion_matrix(y_test, y_pred)

In [ ]:
pd.DataFrame(np.vstack(cm))

,0,1
0,335510,6066
1,6053,301
2,340667,3582
3,3556,125
4,335996,5642
...,...,...
159,2299,64
160,342277,2830
161,2738,85
162,341135,3372


In [49]:
 recall = recall_score(y_test, y_pred, average=None)

In [56]:
precision_score(y_test, y_pred, average='micro')

0.08540622957961229

In [50]:
max(recall)

0.352317880794702

In [51]:
min(recall)

0.0

In [58]:
f1_score(y_test, y_pred, average='micro')

0.08540622957961229

In [44]:
text_representation = tree.export_text(clf)
print(text_representation)

|--- feature_0 <= 6.50
|   |--- feature_2 <= 0.43
|   |   |--- feature_0 <= 1.50
|   |   |   |--- feature_1 <= 0.39
|   |   |   |   |--- feature_2 <= 0.21
|   |   |   |   |   |--- feature_2 <= 0.00
|   |   |   |   |   |   |--- class: sleep
|   |   |   |   |   |--- feature_2 >  0.00
|   |   |   |   |   |   |--- feature_1 <= 0.30
|   |   |   |   |   |   |   |--- feature_2 <= 0.01
|   |   |   |   |   |   |   |   |--- feature_0 <= 0.50
|   |   |   |   |   |   |   |   |   |--- feature_2 <= 0.00
|   |   |   |   |   |   |   |   |   |   |--- class: opera
|   |   |   |   |   |   |   |   |   |--- feature_2 >  0.00
|   |   |   |   |   |   |   |   |   |   |--- class: sleep
|   |   |   |   |   |   |   |   |--- feature_0 >  0.50
|   |   |   |   |   |   |   |   |   |--- feature_3 <= 8.50
|   |   |   |   |   |   |   |   |   |   |--- class: new-age
|   |   |   |   |   |   |   |   |   |--- feature_3 >  8.50
|   |   |   |   |   |   |   |   |   |   |--- class: opera
|   |   |   |   |   |   |   |--- featur